In [1]:
# tube_hammer_dynamics.py
"""
STEP 없이 파라미터로 직접 생성한 '연질 금속 튜브(실링된 고형 원통)'에
해머 임팩트를 가하고 변위-시간 응답을 애니메이션(GIF)으로 저장합니다.
"""

# ────────────────────────────── 사용자 정의 구역 ──────────────────────────────
L      = 0.10     # 튜브 길이 [m]
R_out  = 0.015    # 외경 반지름 [m]
mesh_h = 0.002    # 목표 요소 크기 [m]

E      = 50e9     # Young's modulus [Pa]   (연질 알루미늄 ≈ 50 GPa)
nu     = 0.33     # Poisson's ratio
rho    = 2700     # 밀도 [kg/m³]

F_max  = 4e3      # 해머가 주는 최대 힘 [N] (단면적 평균 하중 ≈ 5 MPa)
t_imp  = 0.001    # 임팩트 지속 시간(반사인 펄스) [s]

t_end  = 3*t_imp  # 해석 끝나는 시점 [s]
n_steps = 150     # 타임스텝 수
scale  = 1000     # PyVista 변위 과장 계수
# ─────────────────────────────────────────────────────────────────────────────

# 0. 기본 import
from mpi4py import MPI
import gmsh, meshio, numpy as np
from dolfinx import mesh, fem, io, plot
import ufl, petsc4py.PETSc as PETSc
import pyvista as pv

# 1. 지오메트리 & 메시 (gmsh)
gmsh.initialize()
gmsh.model.add("tube")
axis  = gmsh.model.occ.addCylinder(0, 0, 0, 0, 0, L, R_out)
gmsh.model.occ.synchronize()
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", mesh_h)
gmsh.model.mesh.generate(3)
gmsh.write("tube.msh")
gmsh.finalize()

# 2. msh → XDMF
meshio.write("tube.xdmf", meshio.read("tube.msh"))

# 3. dolfinx FEA
with io.XDMFFile(MPI.COMM_WORLD, "tube.xdmf", "r") as xdmf:
    domain, = xdmf.read_mesh(name="Grid")

V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))

# 재료 상수
mu, lmbda = E/(2*(1+nu)), E*nu/((1+nu)*(1-2*nu))

def sigma(u):
    eps = ufl.sym(ufl.grad(u))
    return 2*mu*eps + lmbda*ufl.tr(eps)*ufl.Identity(3)

u  = fem.Function(V, name="u")
v  = ufl.TestFunction(V)
du = ufl.TrialFunction(V)

# 질량·강성
dx = ufl.dx
a_K = ufl.inner(sigma(du), ufl.grad(v))*dx
a_M = rho*ufl.inner(du, v)*dx

# 고정(튜브 아랫면 z=0)
bottom_facets = mesh.locate_entities_boundary(
    domain, 2, lambda x: np.isclose(x[2], 0.0, atol=1e-9))
bc = fem.dirichletbc(
    PETSc.ScalarType(0), fem.locate_dofs_topological(V, 2, bottom_facets), V, component=None)

# 임팩트 하중(윗면 z=L) – 반사인 파형
top_facets = mesh.locate_entities_boundary(
    domain, 2, lambda x: np.isclose(x[2], L, atol=1e-9))
dS = ufl.Measure("ds", domain=domain, subdomain_data=None)

A_top = np.pi*R_out**2  # 단면적
def f_t(t):
    return (F_max/A_top)*np.sin(np.pi*t/t_imp) if t <= t_imp else 0.0

# 시간 적분(Newmark 평균가속 β=0.25, γ=0.5)
dt = t_end / n_steps
beta, gamma = 0.25, 0.5
u_n  = fem.Function(V); v_n  = fem.Function(V); a_n  = fem.Function(V)

# 어셈블리 (PETSc 매트릭스)
K = fem.petsc.assemble_matrix(fem.form(a_K), bcs=[bc])
K.assemble()
M = fem.petsc.assemble_matrix(fem.form(a_M), bcs=[])
M.assemble()
A = M.copy(); A.axpy(beta*dt*dt, K, False)  # 유효 강성행렬

solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType("preonly"); solver.getPC().setType("lu")

# 결과 저장 준비
if domain.comm.rank == 0:
    vtm = pv.Plotter(off_screen=True)
    gif = vtm.open_gif("tube_impact.gif", fps=24)

with io.XDMFFile(domain.comm, "u_t.xdmf", "w",
                 encoding=io.XDMFFile.Encoding.HDF5) as xfile:
    xfile.write_mesh(domain)
    for step in range(n_steps+1):
        t = step*dt

        # 외력 벡터
        traction = fem.Constant(domain, (0, 0, -f_t(t)))
        L_ext = ufl.inner(traction, v)*dS
        b = fem.petsc.assemble_vector(fem.form(L_ext))
        fem.apply_lifting(b, [fem.form(a_K)], bcs=[[bc]])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.set_bc(b, [bc])

        # 유효 하중
        rhs = b.copy()
        # M·(ü_term)     + M·(u̇_term)   + M·(u_term)
        rhs.axpy(1.0, M.matMult(a_n.vector))
        rhs.axpy(dt,  M.matMult(v_n.vector))
        rhs.axpy((0.5 - beta)*dt*dt, K.matMult(u_n.vector))

        # 선형 해 풀기
        u.vector[:] = 0.0
        solver.solve(rhs, u.vector)
        u.vector.ghostUpdate()

        # 속도·가속도 업데이트
        a_n.vector[:] = (u.vector - u_n.vector - dt*v_n.vector -
                         (0.5 - beta)*dt*dt*a_n.vector) / (beta*dt*dt)
        v_n.vector[:] = v_n.vector + dt*((1 - gamma)*a_n.vector + gamma*a_n.vector)
        # 스테이트 이동
        u_n.x.array[:] = u.x.array

        # 파일 + GIF 기록 (rank0만)
        xfile.write_function(u, t)
        if domain.comm.rank == 0:
            grid = plot.create_vtk_mesh(V)[0]
            grid.point_data["disp"] = u.x.array.reshape((-1, 3))*scale
            warped = grid.warp_by_vector("disp", factor=1)
            vtm.add_mesh(warped, scalars="disp", show_edges=False,
                         cmap="turbo", clim=[0, scale*1e-3])
            vtm.write_frame()
            vtm.clear()

    if domain.comm.rank == 0:
        vtm.close()


ModuleNotFoundError: No module named 'dolfinx'